*프로그램 설치 : conda install -c anaconda py-xgboot  
*프로그램 설치2
 https://www.lfd.uci.edu/~gohlke/pythonlibs/
 pip install xgboost-1.0.2-cp37-cp37m-win_amd64.whl
*Decision Tree 여러 개를 결합하여 동작하는 알고리즘
*트리기반 앙상블 알고리즘 중에 최근에 가장 좋은 성능을 보여주고 있는 알고리즘
*GradientBootsing에 기반하고 있지만 좀 더 빠른 속도를 보장하고 Overfitting을 제어할 수 있다.
*교차검증 기능이 내장되어 있어 성능이 더 이상 증가하지 않으면 자체적으로 학습 종료시킨다.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
import util
import pandas as pd
import numpy as np

df = pd.read_csv('myfile_eng.csv', encoding='euc-kr')
df = df[df.dan < 8]
stnd_cols = ['tms','days','race']
odds_cols = ['dan','yun1','yun2','san','bok']
result_cols = ['1nd','2nd','3nd']

df_x = df.drop(stnd_cols+odds_cols+result_cols, axis=1) #기준 칼럼 삭제
df_y = df['1nd'] #(*)

X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.25, random_state=0) 

*하이퍼파리미터 설정
*objective : 손실함수, defalut: 'reg:linear'
  *'binary:logistic' : 이항분류 로지스틱 
  *'multi:softmax' : 다항분류 소프트맥스
  *'multi:softprob' : 각 분류에 속할 확률 반환
*eta : learning_rate와 같은 동작을하는 파라미터, default:0.3
  *0과 1사이의 값, 반복적으로 학습할 때 학습 속도 결정
*num_boost_rounds : 내부적으로 사용할 tree의 갯수, n_estimeter와 같은 역할
  *다른 파라미터에 영향을 많이 받기 때문에, 다른 파리미터가 수정되면 같이 변경해야 한다.

In [ ]:
import xgboost as xgb
model = xgb.XGBClassifier(objective="reg:linear", random_state=42)
# model = xgb.XGBClassifier(objective='binary:softprob', eta=0.3, num_boost_rounds=500)
model.fit(X_train, y_train)
print('score: %.2f' % model.score(X_test, y_test))

In [ ]:
print('feature_importances: ', model.feature_importances_) #feature가 분류에 얼마나 사용됐는지 확인

In [ ]:
#feature_importances_ : feature가 알고리즘에서 얼마나 사용되었는지
df_features = pd.DataFrame([X_train.columns,model.feature_importances_])
util.log(df_features, 'df_features1')
df_features = df_features.T.rename(columns={0:'name', 1:'imp'})
# df_features = df_features.rename(columns={0:'name', 1:'imp'})
util.log(df_features, 'df_features2')

In [ ]:
#feature의 사용빈도가 없는 컬럼을
df_features.imp = df_features.imp.astype('float')
util.log(df_features.imp.describe(), 'df_features.imp.describe()')
# df_temp = df_features[df_features.imp < 0.002]
# df_temp2 = df_temp.drop([81,116], axis=0)
util.log(df_temp, 'df_temp')

In [ ]:
stnd_cols = ['tms','days','race']
odds_cols = ['dan','yun1','yun2','san','bok']
result_cols = ['1nd','2nd','3nd']
low_cols = list(df_temp.name)

df_x = df.drop(stnd_cols+odds_cols+result_cols+low_cols, axis=1) #기준 칼럼 삭제
util.log(df_x)
df_y = df['1nd'] #(*)

X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.25, random_state=0) 

In [ ]:
model.fit(X_train, y_train)
print('score: %.2f' % model.score(X_test, y_test))